<a href="https://colab.research.google.com/github/Taedriel/ZSL-v2/blob/wordEmbedding/WordsEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Kingston/ZSL-v2/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Kingston/ZSL-v2


In [2]:
!pip install transformers gensim wget

In [3]:
!git pull

remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/Taedriel/ZSL-v2
   dc93e47d..004441d7  wordEmbedding -> origin/wordEmbedding
Updating dc93e47d..004441d7
Fast-forward
 converter.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [4]:
from converter import *
model = BERTModel(["king", "queen", "man", " woman"])

# model.importTagList("en-basic")
model.convert()
model.export("en-basic-Embedded.csv")

# model.computeCoSim()
# model.simBetween("cat", "dog")

man = model.get_embedding_of("man")[1]
woman = model.get_embedding_of("woman")[1]

king = model.get_embedding_of("king")[1]

totest = king.sub(man).add(woman)
model.get_nearest_embedding_of(totest, 3)

model.export_sim_matrix("sim_matrix.csv")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:root:Starting converting tokens...
INFO:root:[0] Number of tokens: 1
INFO:root:20.0% completed
INFO:root:[1] Number of tokens: 1
INFO:root:50.0% compl